In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW


# =========================================================
# 0) 설정 (너 환경에 맞게 여기만 바꾸면 됨)
# =========================================================
MODEL_NAME = "kakaobank/kf-deberta-base"

LABELED_CSV = "/content/drive/MyDrive/labelling_end2.csv"     # ✅ -1/0/1만 있는 학습용 라벨 csv
PREDICT_CSV = "./to_predict.csv"        # ✅ 예측할 csv (없으면 학습 파일로 예시 예측)
OUT_PRED_CSV = "/content/drive/MyDrive/predicted_with_neutral.csv"

SAVE_DIR = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real"

TITLE_COL = "제목"
BODY_COL  = "본문"
LABEL_COL = "label"  # ✅ 라벨 컬럼명 (-1 또는 1)

MAX_LEN = 256        # 제목+본문이면 256 권장 (제목만이면 64)
EPOCHS = 2
LR = 2e-5
TRAIN_BS = 8         # DeBERTa 무거움: 8~16 추천
VAL_BS = 16
SEED = 42

# ✅ 중립(0) 임계값: 예측 때만 사용
TAU_CONF = 0.60      # max(prob) < 0.60 -> 중립
DELTA_MARGIN = 0.20  # |p_pos - p_neg| < 0.20 -> 중립


# =========================================================
# 1) 유틸
# =========================================================
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def softmax_np(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=-1, keepdims=True)

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# =========================================================
# 2) 라벨 데이터 로드
# =========================================================
df = pd.read_csv(LABELED_CSV, encoding="utf-8-sig")

if TITLE_COL not in df.columns:
    raise KeyError(f"'{TITLE_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

if BODY_COL not in df.columns:
    df[BODY_COL] = ""

if LABEL_COL not in df.columns:
    raise KeyError(f"'{LABEL_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

# 라벨 정리 (문자열/1.0/-1.0 방지)
df[LABEL_COL] = (
    df[LABEL_COL].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
)
df[LABEL_COL] = pd.to_numeric(df[LABEL_COL], errors="coerce")

# -1, 1만 남기기
df = df[df[LABEL_COL].isin([-1, 0, 1])].copy()

# ✅ 2클래스 매핑 (이게 네가 원한 "매핑 코드") -> Updated for 3-class indices
label_map = {-1: 0, 0 : 1, 1: 2}        # NEG=0, NEUTRAL=1, POS=2
inv_label_map = {0: -1, 1: 0, 2: 1}    # 예측값을 -1/0/1로 되돌릴 때

df["label_id"] = df[LABEL_COL].map(label_map).astype(int)

print("label 분포:\n", df[LABEL_COL].value_counts())
print("label_id 분포:\n", df["label_id"].value_counts())


# 입력 텍스트 (제목+본문)
df["text"] = df[TITLE_COL].astype(str) + "\n\n" + df[BODY_COL].astype(str)

train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label_id"]
)
print("train:", len(train_df), "val:", len(val_df))


# =========================================================
# 3) Dataset / DataLoader
# =========================================================
class NewsDataset(Dataset):
    def __init__(self, df_):
        self.texts = df_["text"].astype(str).tolist()
        self.labels = df_["label_id"].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_loader = DataLoader(NewsDataset(train_df), batch_size=TRAIN_BS, shuffle=True)
val_loader   = DataLoader(NewsDataset(val_df), batch_size=VAL_BS, shuffle=False)


# =========================================================
# 4) 모델 (✅ 카카오뱅크 모델 기반 3클래스)
# =========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
).to(device)

optimizer = AdamW(model.parameters(), lr=LR)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)


# =========================================================
# 5) 학습 / 평가
# =========================================================
def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"[TRAIN] Epoch {epoch+1}/{EPOCHS}")

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=total_loss / max(1, len(pbar)))

    return total_loss / max(1, len(train_loader))

def evaluate():
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            labels = batch["labels"].cpu().numpy()
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())

    print("[VAL] loss:", total_loss / max(1, len(val_loader)))
    # Updated target_names for 3 classes (0, 1, 2) mapped to (-1, 0, 1)
    print(classification_report(y_true, y_pred, target_names=["NEG(-1)", "NEU(0)", "POS(1)"]))

for epoch in range(EPOCHS):
    tr_loss = train_one_epoch(epoch)
    print("train loss:", tr_loss)
    evaluate()


# =========================================================
# 6) 저장
# =========================================================
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("✅ 파인튜닝 모델 저장 완료:", SAVE_DIR)

device: cuda
label 분포:
 label
 1    20000
-1    20000
 0    10000
Name: count, dtype: int64
label_id 분포:
 label_id
2    20000
0    20000
1    10000
Name: count, dtype: int64
train: 40000 val: 10000


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at kakaobank/kf-deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[TRAIN] Epoch 1/2: 100%|██████████| 5000/5000 [23:46<00:00,  3.51it/s, loss=0.179]


train loss: 0.17946218692458935
[VAL] loss: 0.06765469159213826
              precision    recall  f1-score   support

     NEG(-1)       0.99      0.99      0.99      4000
      NEU(0)       0.97      0.95      0.96      2000
      POS(1)       0.98      0.99      0.99      4000

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



[TRAIN] Epoch 2/2: 100%|██████████| 5000/5000 [23:46<00:00,  3.50it/s, loss=0.0367]


train loss: 0.03668328886179952
[VAL] loss: 0.03196981580597349
              precision    recall  f1-score   support

     NEG(-1)       0.99      1.00      0.99      4000
      NEU(0)       0.98      0.97      0.98      2000
      POS(1)       0.99      0.99      0.99      4000

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

✅ 파인튜닝 모델 저장 완료: /content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real


In [ ]:
# 예측

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰주세요)
# ==========================================
MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real" # 저장된 경로
INPUT_CSV = "/content/drive/MyDrive/news_clean/KB금융_clean.csv"  # 예측하고 싶은 파일
OUTPUT_CSV = "/content/drive/MyDrive/KB금융_predicted_results.csv" # 결과 저장 경로

MAX_LEN = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. 모델 및 토크나이저 로드
# ==========================================
print("모델 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()

# ==========================================
# 3. 데이터 로드 및 전처리
# ==========================================
df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

# 제목과 본문 합치기 (학습 때와 동일한 포맷)
# 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
if "본문" not in df.columns:
    df["본문"] = ""

texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

# ==========================================
# 4. 예측 (Inference)
# ==========================================
results = []
inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

print(f"{len(texts)}개의 기사 분석 시작...")

with torch.no_grad():
    for text in tqdm(texts):
        # 토큰화
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN,
            padding="max_length"
        ).to(device)

        # 모델 추론
        outputs = model(**inputs)
        logits = outputs.logits

        # 확률값 계산 (Softmax)
        probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

        # 가장 높은 확률의 인덱스 선택
        pred_id = np.argmax(probs)
        pred_label = inv_label_map[pred_id]

        # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
        results.append({
            "pred_label": pred_label,
            "prob_neg": round(probs[0], 4),
            "prob_neu": round(probs[1], 4),
            "prob_pos": round(probs[2], 4)
        })

# ==========================================
# 5. 결과 합치기 및 저장
# ==========================================
pred_df = pd.DataFrame(results)
final_df = pd.concat([df, pred_df], axis=1)

# 감성을 한글로 보고 싶다면 추가
final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


10353개의 기사 분석 시작...


100%|██████████| 10353/10353 [04:00<00:00, 43.04it/s]


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/KB금융_predicted_results.csv


In [ ]:
df_pred = pd.read_csv('/content/drive/MyDrive/predicted_results.csv')

In [ ]:
df_pred['sentiment'].value_counts()

,count
sentiment,
중립,1798
호재(긍정),506
악재(부정),179


In [ ]:
samsungbio = pd.read_csv('/content/drive/MyDrive/삼성바이오로직스_predicted_results.csv')

In [ ]:
samsungbio['sentiment'].value_counts()

,count
sentiment,
중립,4207
호재(긍정),3626
악재(부정),1075


In [ ]:
pd.read_csv('/content/drive/MyDrive/KB금융_predicted_results.csv')['sentiment'].value_counts()

,count
sentiment,
중립,8149
호재(긍정),1531
악재(부정),673


In [ ]:
# 추가학습

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW


# =========================================================
# 0) 설정 (너 환경에 맞게 여기만 바꾸면 됨)
# =========================================================
MODEL_NAME = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real/"

LABELED_CSV = "/content/drive/MyDrive/rest_label.csv"     # ✅ -1/1만 있는 학습용 라벨 csv
PREDICT_CSV = "./to_predict.csv"        # ✅ 예측할 csv (없으면 학습 파일로 예시 예측)
OUT_PRED_CSV = "/content/drive/MyDrive/predicted_with_neutral.csv"

SAVE_DIR = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus"

TITLE_COL = "제목"
BODY_COL  = "본문"
LABEL_COL = "label"  # ✅ 라벨 컬럼명 (-1 또는 1)

MAX_LEN = 256        # 제목+본문이면 256 권장 (제목만이면 64)
EPOCHS = 2
LR = 2e-5
TRAIN_BS = 8         # DeBERTa 무거움: 8~16 추천
VAL_BS = 16
SEED = 42

# ✅ 중립(0) 임계값: 예측 때만 사용
TAU_CONF = 0.60      # max(prob) < 0.60 -> 중립
DELTA_MARGIN = 0.20  # |p_pos - p_neg| < 0.20 -> 중립


# =========================================================
# 1) 유틸
# =========================================================
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def softmax_np(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=-1, keepdims=True)

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# =========================================================
# 2) 라벨 데이터 로드 + ✅ 매핑(-1/1 -> label_id 0/1)
# =========================================================
df = pd.read_csv(LABELED_CSV, encoding="utf-8-sig")

if TITLE_COL not in df.columns:
    raise KeyError(f"'{TITLE_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

if BODY_COL not in df.columns:
    df[BODY_COL] = ""

if LABEL_COL not in df.columns:
    raise KeyError(f"'{LABEL_COL}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

# 라벨 정리 (문자열/1.0/-1.0 방지)
df[LABEL_COL] = (
    df[LABEL_COL].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
)
df[LABEL_COL] = pd.to_numeric(df[LABEL_COL], errors="coerce")

# -1, 1만 남기기
df = df[df[LABEL_COL].isin([-1, 1])].copy()

# ✅ 2클래스 매핑 (이게 네가 원한 "매핑 코드")
label_map = {-1: 0, 1: 2}        # NEG=0, POS=1
inv_label_map = {0: -1, 2: 1}    # 예측값을 -1/1로 되돌릴 때

df["label_id"] = df[LABEL_COL].map(label_map).astype(int)

print("label 분포:\n", df[LABEL_COL].value_counts())
print("label_id 분포:\n", df["label_id"].value_counts())


# 입력 텍스트 (제목+본문)
df["text"] = df[TITLE_COL].astype(str) + "\n\n" + df[BODY_COL].astype(str)

train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["label_id"]
)
print("train:", len(train_df), "val:", len(val_df))


# =========================================================
# 3) Dataset / DataLoader
# =========================================================
class NewsDataset(Dataset):
    def __init__(self, df_):
        self.texts = df_["text"].astype(str).tolist()
        self.labels = df_["label_id"].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_loader = DataLoader(NewsDataset(train_df), batch_size=TRAIN_BS, shuffle=True)
val_loader   = DataLoader(NewsDataset(val_df), batch_size=VAL_BS, shuffle=False)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# =========================================================
# 4) 모델 (✅ 카카오뱅크 모델 기반 2클래스)
# =========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
).to(device)

optimizer = AdamW(model.parameters(), lr=LR)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)


# =========================================================
# 5) 학습 / 평가
# =========================================================
def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"[TRAIN] Epoch {epoch+1}/{EPOCHS}")

    for batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=total_loss / max(1, len(pbar)))

    return total_loss / max(1, len(train_loader))

def evaluate():
    model.eval()
    y_true, y_pred = [], []
    total_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            labels = batch["labels"].cpu().numpy()
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            y_true.extend(labels.tolist())
            y_pred.extend(preds.tolist())

    print("[VAL] loss:", total_loss / max(1, len(val_loader)))
    print(classification_report(y_true, y_pred, labels=[0, 1, 2], target_names=["NEG(-1)", "NEU(0)", "POS(1)"]))

for epoch in range(EPOCHS):
    tr_loss = train_one_epoch(epoch)
    print("train loss:", tr_loss)
    evaluate()


# =========================================================
# 6) 저장
# =========================================================
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("✅ 파인튜닝 모델 저장 완료:", SAVE_DIR)


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real/' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


device: cuda
label 분포:
 label
 1    29122
-1    29122
Name: count, dtype: int64
label_id 분포:
 label_id
2    29122
0    29122
Name: count, dtype: int64
train: 46595 val: 11649


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real/' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
[TRAIN] Epoch 1/2: 100%|██████████| 5825/5825 [27:36<00:00,  3.52it/s, loss=0.0135]


train loss: 0.013461193525459833


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

[VAL] loss: 0.005863204843344135
              precision    recall  f1-score   support

     NEG(-1)       1.00      1.00      1.00      5825
      NEU(0)       0.00      0.00      0.00         0
      POS(1)       1.00      1.00      1.00      5824

    accuracy                           1.00     11649
   macro avg       0.66      0.66      0.66     11649
weighted avg       1.00      1.00      1.00     11649



[TRAIN] Epoch 2/2: 100%|██████████| 5825/5825 [27:42<00:00,  3.50it/s, loss=0.00371]


train loss: 0.0037067251248204484
[VAL] loss: 0.00292770867918641
              precision    recall  f1-score   support

     NEG(-1)       1.00      1.00      1.00      5825
      NEU(0)       0.00      0.00      0.00         0
      POS(1)       1.00      1.00      1.00      5824

    accuracy                           1.00     11649
   macro avg       0.67      0.67      0.67     11649
weighted avg       1.00      1.00      1.00     11649



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

✅ 파인튜닝 모델 저장 완료: /content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus


In [ ]:
MODEL_NAME = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real/"
import os
print(os.path.exists(MODEL_NAME)) # True가 나와야 합니다.
print(os.listdir(MODEL_NAME))

True
['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json']


In [ ]:
# 예측

In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰주세요)
# ==========================================
MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus" # 저장된 경로
INPUT_CSV = "/content/drive/MyDrive/news_clean/KB금융_clean.csv"  # 예측하고 싶은 파일
OUTPUT_CSV = "/content/drive/MyDrive/KB금융_predicted_results_plus.csv" # 결과 저장 경로

MAX_LEN = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. 모델 및 토크나이저 로드
# ==========================================
print("모델 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()

# ==========================================
# 3. 데이터 로드 및 전처리
# ==========================================
df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

# 제목과 본문 합치기 (학습 때와 동일한 포맷)
# 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
if "본문" not in df.columns:
    df["본문"] = ""

texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

# ==========================================
# 4. 예측 (Inference)
# ==========================================
results = []
inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

print(f"{len(texts)}개의 기사 분석 시작...")

with torch.no_grad():
    for text in tqdm(texts):
        # 토큰화
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN,
            padding="max_length"
        ).to(device)

        # 모델 추론
        outputs = model(**inputs)
        logits = outputs.logits

        # 확률값 계산 (Softmax)
        probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

        # 가장 높은 확률의 인덱스 선택
        pred_id = np.argmax(probs)
        pred_label = inv_label_map[pred_id]

        # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
        results.append({
            "pred_label": pred_label,
            "prob_neg": round(probs[0], 4),
            "prob_neu": round(probs[1], 4),
            "prob_pos": round(probs[2], 4)
        })

# ==========================================
# 5. 결과 합치기 및 저장
# ==========================================
pred_df = pd.DataFrame(results)
final_df = pd.concat([df, pred_df], axis=1)

# 감성을 한글로 보고 싶다면 추가
final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


10353개의 기사 분석 시작...


100%|██████████| 10353/10353 [04:02<00:00, 42.62it/s]


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/KB금융_predicted_results_plus.csv


In [ ]:
df_pred_plus = pd.read_csv('/content/drive/MyDrive/BGF리테일_predicted_results_plus.csv')
df_pred_plus

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150111,"BGF리테일, 브랜드 비전 선포식 개최...새 슬로건 공개",BGF리테일은 지난 9일 ‘브랜드 비전 선포식’을 갖고 BGF와 CU의 슬로건을 새...,282330,1,0.0151,0.0001,0.9848,호재(긍정)
1,20150112,[동정] 홍석조 BGF리테일 회장 비전 선포식 外,홍석조 BGF리테일 회장 비전 선포식홍석조(62 사진) BGF리테일 회장은 지난 9...,282330,-1,0.8263,0.0020,0.1717,악재(부정)
2,20150114,"BGF리테일, 4Q 실적 눈높이 충족 예상 투자의견↑ 목표가↑-HMC",[ 이민하 기자 ] HMC투자증권은 14일 BGF리테일에 대해 지난 4분기 실적이 ...,282330,1,0.0587,0.0006,0.9407,호재(긍정)
3,20150114,"BGF리테일, 양호한 실적이 재평가 견인-HMC",HMC투자증권은 14일 BGF리테일이 속한 편의점 업종이 올해도 유통채널 중 가장 ...,282330,1,0.0118,0.0003,0.9879,호재(긍정)
4,20150114,"BGF리테일, 편의점 성장 기대감 투자의견↑-HMC",[이데일리 김인경 기자] HMC투자증권은 BGF리테일(027410)의 실적성장이 나...,282330,1,0.0482,0.0006,0.9512,호재(긍정)
...,...,...,...,...,...,...,...,...,...
2478,20241202,"BGF리테일, 수익성 개선 구간 진입 ""밸류에이션 상단은 제한적""-하나증권",[머니투데이 천현정 기자] 하나증권은 BGF리테일이 수익성 개선 구간에 진입했으나 ...,282330,1,0.0143,0.0002,0.9855,호재(긍정)
2479,20241218,"민승배 BGF리테일 대표, 한국유통대상 동탑산업훈장 수상",편의점 CU를 운영하는 BGF리테일은 민승배 대표가 이달 17일 대한상공회의소에서 ...,282330,1,0.0922,0.0060,0.9018,호재(긍정)
2480,20241218,"민승배 BGF리테일 대표, 제29회 한국유통대상 동탑산업훈장 수상",BGF리테일은 민승배 대표가 이달 17일 대한상공회의소에서 열린 제29회 한국유통대...,282330,1,0.0236,0.0023,0.9741,호재(긍정)
2481,20241218,"[단독] 고속도로 휴게소 진출한 BGF리테일 ‘함박웃음’, 얼마나 벌길래? [언박싱]","BGF리테일, 고속도로 5개 휴게시설 운영권 낙찰 \n年매출 776억원 예상 일반 ...",282330,1,0.0000,0.0000,1.0000,호재(긍정)


In [ ]:
df_pred_plus['sentiment'].value_counts()

,count
sentiment,
호재(긍정),1644
악재(부정),839


In [ ]:
df_pred_plus = pd.read_csv('/content/drive/MyDrive/KB금융_predicted_results_plus.csv')
df_pred_plus

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150102,"[신년사] 윤종규 KB금융 회장 ""소신껏 일하는 분위기 만들 것""",윤종규 KB금융그룹 회장 겸 국민은행장[사진=KB금융지주 제공] \n \n \n아주...,105560,1,0.4142,0.0064,0.5794,호재(긍정)
1,20150102,"윤종규 KB금융 회장, 경영진과 함께 1박2일 워크샵 진행",[이투데이] 박선현 기자(sunhyun@etoday.co.kr)윤종규 KB금융 회장...,105560,-1,0.7008,0.0022,0.2970,악재(부정)
2,20150102,윤종규 KB금융 회장 “신바람 나는 일터 만들자”,윤종규(사진) KB금융지주 회장은 2일 오전 여의도 국민은행 본점 강당에서 열린 취...,105560,1,0.0268,0.0004,0.9728,호재(긍정)
3,20150102,"[신년사] 윤종규 KB금융 회장 ""실패 두려워 말고, 일하는 KB 만들자""","[아시아경제 김대섭 기자] 윤종규 KB금융그룹 회장은 2일 신년사에서 ""그동안 임직...",105560,1,0.2360,0.0012,0.7629,호재(긍정)
4,20150102,"[신년사] 윤종규 KB금융 회장 ""이제는 쇄신을 행동으로 옮길 때""","[뉴스핌=노희준 기자] 윤종규 KB금융지주 회장 겸 국민은행장(사진)은 2일 ""이제...",105560,1,0.0688,0.0006,0.9306,호재(긍정)
...,...,...,...,...,...,...,...,...,...
10348,20241226,"KB금융, 조직 개편 경영진 인사 “효율 혁신” 초점",KB금융지주는 26일 정기 조직개편과 경영진 인사를 단행했다고 밝혔다. \n \n ...,105560,-1,1.0000,0.0000,0.0000,악재(부정)
10349,20241226,"KB금융, 이재근 글로벌부문장 이창권 디지털부문장 선임",KB금융지주는 조직 개편과 경영진 인사를 단행했다고 26일 밝혔다.\n\n이재근 현...,105560,-1,0.7958,0.0009,0.2034,악재(부정)
10350,20241227,"KB금융, ‘혁신성장 효율경영’ 조직개편ㆍ경영진 인사 이재근 이창권 부문장 기용",[이투데이] 손희정 기자 (sonhj1220@etoday.co.kr)\n\nKB금융...,105560,-1,0.6049,0.0017,0.3934,악재(부정)
10351,20241227,"KB금융, 조직개편 인사 실시 상생 효율조직 미래성장 추진",KB금융지주는 정기 조직개편과 경영진 인사를 실시했다고 27일 밝혔다. \n\n이번...,105560,-1,1.0000,0.0000,0.0000,악재(부정)


In [ ]:
df_pred_plus['sentiment'].value_counts()

,count
sentiment,
호재(긍정),6668
악재(부정),3685


In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 (학습 때와 동일하게 맞춰주세요)
# ==========================================
MODEL_PATH = "/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus" # 저장된 경로
INPUT_CSV = "/content/drive/MyDrive/news_clean/KB금융_clean.csv"  # 예측하고 싶은 파일
OUTPUT_CSV = "/content/drive/MyDrive/KB금융_predicted_results_plus.csv" # 결과 저장 경로

MAX_LEN = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. 모델 및 토크나이저 로드
# ==========================================
print("모델 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
model.eval()

# ==========================================
# 3. 데이터 로드 및 전처리
# ==========================================
df = pd.read_csv(INPUT_CSV, encoding="utf-8-sig")

# 제목과 본문 합치기 (학습 때와 동일한 포맷)
# 만약 본문 컬럼이 없다면 제목만 사용하도록 처리
if "본문" not in df.columns:
    df["본문"] = ""

texts = (df["제목"].astype(str) + "\n\n" + df["본문"].astype(str)).tolist()

# ==========================================
# 4. 예측 (Inference)
# ==========================================
results = []
inv_label_map = {0: -1, 1: 0, 2: 1} # 0=부정, 1=중립, 2=긍정

print(f"{len(texts)}개의 기사 분석 시작...")

with torch.no_grad():
    for text in tqdm(texts):
        # 토큰화
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_LEN,
            padding="max_length"
        ).to(device)

        # 모델 추론
        outputs = model(**inputs)
        logits = outputs.logits

        # 확률값 계산 (Softmax)
        probs = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]

        # 가장 높은 확률의 인덱스 선택
        pred_id = np.argmax(probs)
        pred_label = inv_label_map[pred_id]

        # 결과 저장 (확률값도 함께 저장하면 분석에 용이함)
        results.append({
            "pred_label": pred_label,
            "prob_neg": round(probs[0], 4),
            "prob_neu": round(probs[1], 4),
            "prob_pos": round(probs[2], 4)
        })

# ==========================================
# 5. 결과 합치기 및 저장
# ==========================================
pred_df = pd.DataFrame(results)
final_df = pd.concat([df, pred_df], axis=1)

# 감성을 한글로 보고 싶다면 추가
final_df['sentiment'] = final_df['pred_label'].map({-1: "악재(부정)", 0: "중립", 1: "호재(긍정)"})

final_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 예측 완료! 결과 저장됨: {OUTPUT_CSV}")

모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/news_sentiment_ft_kakaobank_2class_real_plus' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


10353개의 기사 분석 시작...


100%|██████████| 10353/10353 [04:02<00:00, 42.63it/s]


✅ 예측 완료! 결과 저장됨: /content/drive/MyDrive/KB금융_predicted_results_plus.csv


In [ ]:
df_pred_plus2 = pd.read_csv('/content/drive/MyDrive/NAVER_predicted_results_plus.csv')
df_pred_plus2

,일자,제목,본문,ticker,pred_label,prob_neg,prob_neu,prob_pos,sentiment
0,20150105,"외인, 지난해 5.8조 순매수..삼성電 사고 NAVER 팔았다",지난해 국내주식시장에서 외국인 투자자가 5조8000억원을 순매수한 것으로 집계됐다....,35420,1,0.0170,0.0005,0.9825,호재(긍정)
1,20150105,外人 작년 최대매수-매도종목‘삼성전자-NAVER’,지난해 외국인이 국내 주식시장에서 6조원 가까이 순매수한 것으로 나타났다. 삼성전자...,35420,1,0.0125,0.0006,0.9869,호재(긍정)
2,20150105,"[Hot-Line] ""NAVER, 지난해 4분기 매출 기대치 부합할 것""",하이투자증권은 5일 NAVER의 지난해 4분기 실적이 시장 기대에 부합할 것이라고 ...,35420,1,0.0746,0.0016,0.9238,호재(긍정)
3,20150105,"[특징주]NAVER, 외국계 매수세에 강세",[아시아경제 서소정 기자]NAVER가 외국계 매수세에 힘입어 강세다.5일 오전 10...,35420,1,0.0000,0.0000,1.0000,호재(긍정)
4,20150105,"[특징주] NAVER, 강세...외국계 매수세",[뉴스핌=이영기 기자] NAVER 주식이 외국계 매수세 영향으로 강세다. \n \n...,35420,1,0.0000,0.0000,1.0000,호재(긍정)
...,...,...,...,...,...,...,...,...,...
3215,20241212,"[리포트 브리핑]NAVER, '더 높은 곳으로!' 목표가 280,000원 - 현대차증권",[서울=뉴스핌] 로보뉴스 = 현대차증권에서 12일 NAVER(035420)에 대해 ...,35420,1,0.0000,0.0000,1.0000,호재(긍정)
3216,20241214,[유안타證 주간추천주]NAVER 엔씨소프트 슈프리마,"[이데일리 이정현 기자] △NAVER(035420)\n\n-AI, LY 와 관련한 ...",35420,1,0.0000,0.0000,1.0000,호재(긍정)
3217,20241218,"[리포트 브리핑]NAVER, '젊어지는 네이버' 목표가 290,000원 - 유안타증권",[서울=뉴스핌] 로보뉴스 = 유안타증권에서 18일 NAVER(035420)에 대해 ...,35420,1,0.0000,0.0000,1.0000,호재(긍정)
3218,20241221,[유안타證 주간추천주]NAVER 엔씨소프트 KT,"[이데일리 이정현 기자]\n\n△NAVER(035420)\n\n-AI, LY 와 관...",35420,1,0.1066,0.0002,0.8932,호재(긍정)


In [ ]:
df_pred_plus2['sentiment'].value_counts()

,count
sentiment,
호재(긍정),2781
악재(부정),439


In [ ]:
df_plus = pd.read_csv('/content/drive/MyDrive/KB금융_predicted_results_plus.csv')

In [ ]:
df_plus['pred_label'].value_counts()

,count
pred_label,
1,6668
-1,3685
